In [1]:
import traitlets
import threading
import numpy as np
import atexit
import cv2 
import logging
import math
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import os

from collections import defaultdict
from io import StringIO
#from matplotlib import pyplot as plt
#import matplotlib; matplotlib.use('Agg')
from PIL import Image
import time


from utils import label_map_util

from utils import visualization_utils as vis_util


class Camera(traitlets.HasTraits):

    value = traitlets.Any()
    width = traitlets.Integer(default_value=224)
    height = traitlets.Integer(default_value=224)
    format = traitlets.Unicode(default_value='bgr8')
    running = traitlets.Bool(default_value=False)
    
    def __init__(self, *args, **kwargs):
        super(Camera, self).__init__(*args, **kwargs)
        if self.format == 'bgr8':
            self.value = np.empty((self.height, self.width, 3), dtype=np.uint8)
        self._running = False
            
    def _read(self):
        """Blocking call to read frame from camera"""
        raise NotImplementedError
        
    def read(self):
        if self._running:
            raise RuntimeError('Cannot read directly while camera is running')
        self.value = self._read()
        return self.value
    
    def _capture_frames(self):
        while True:
            if not self._running:
                break
            self.value = self._read()
            
    @traitlets.observe('running')
    def _on_running(self, change):
        if change['new'] and not change['old']:
            # transition from not running -> running
            self._running = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()
        elif change['old'] and not change['new']:
            # transition from running -> not running
            self._running = False
            self.thread.join()

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
#from .camera import Camera



class USBCamera(Camera):
    
    capture_fps = traitlets.Integer(default_value=30)
    capture_width = traitlets.Integer(default_value=640)
    capture_height = traitlets.Integer(default_value=480)   
    capture_device = traitlets.Integer(default_value=0)
    
    def __init__(self, *args, **kwargs):
        super(USBCamera, self).__init__(*args, **kwargs)
        try:
            self.cap = cv2.VideoCapture(self._gst_str(), cv2.CAP_GSTREAMER)

            re , image = self.cap.read()
            
            if not re:
                raise RuntimeError('Could not read image from camera.')
            
        except:
            raise RuntimeError(
                'Could not initialize camera.  Please see error trace.')

        atexit.register(self.cap.release)
                
    def _gst_str(self):
        return 'v4l2src device=/dev/video{} ! video/x-raw, width=(int){}, height=(int){}, framerate=(fraction){}/1 ! videoconvert !  video/x-raw, format=(string)BGR ! appsink'.format(self.capture_device, self.capture_width, self.capture_height, self.capture_fps)
    
    def _read(self):
        re, image = self.cap.read()
        if re:
            image_resized = cv2.resize(image,(int(self.width),int(self.height)))
            return image_resized
        else:
            raise RuntimeError('Could not read image from camera')

In [3]:
cam=USBCamera(width=640, height=480, capture_width=640, capture_height=480, capture_device=0)

In [4]:
image=cam.read()

In [5]:
print(cam.value.shape)

(480, 640, 3)


In [6]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [7]:
def region_of_interest(edges):
    #returns an image that masks the unnecessary parts which contain surrounding environment in the original image
    height,width=edges.shape
    mask= np.zeros_like(edges)
    polygon=np.array([[(0, height*0.4),(width,height*0.4),(width,height),(0,height),]],np.int32)
    cv2.fillPoly(mask,polygon,255)
    cropped_edges=cv2.bitwise_and(edges,mask)
    return cropped_edges


In [8]:
def detect_line_segments(cropped_edges):
    rho=1;   #distance precision pixel
    angle= np.pi/180 # angulat precision in radians
    min_threshold=10 #minimal of votes
    line_segments=cv2.HoughLinesP(cropped_edges, rho, angle, min_threshold, np.array([]), minLineLength=10, maxLineGap=15)
    #Hough transform returns the line segments
    return line_segments



In [9]:
def make_points(frame, line):
    height, width, _ = frame.shape
    slope, intercept = line
    y1 = height  # bottom of the frame
    y2 = int(y1 * 1 / 2)  # make points from middle of the frame down

    # bound the coordinates within the frame
    x1 = max(-width, min(2 * width, int((y1 - intercept) / slope)))
    x2 = max(-width, min(2 * width, int((y2 - intercept) / slope)))
    return [[x1, y1, x2, y2]]

In [10]:
def average_slope_intercept(frame, line_segments):
    #combines two line segments
    lane_lines=[]
    if line_segments is None:
        logging.info('No line segments detected')
        return lane_lines
    height,width,_=frame.shape
    left_fit= []
    right_fit= []
    boundary= 1/2   #Multiplier that divides the region into left region and right region
    left_region_boundary= width*(1-boundary) #width of left region
    right_region_boundary = width* boundary  #Width of right region
    
    for line_segment in line_segments:
        for x1,y1,x2,y2 in line_segment:
            if x1==x2:   
                #skips the iteration if it detects a vertical line
                continue
            fit= np.polyfit((x1,x2),(y1,y2),1)  
            #returns an array of coefficients of the equation of the line, y=mx+C considering the points on the line
            slope = fit[0]  #slope is the first coefficient   
            intercept= fit[1] # intercept is the second coefficient
            if slope<0:
                if x1< left_region_boundary and x2< left_region_boundary: #Checks if the line is in the left region
                    left_fit.append((slope,intercept))  #Appends points to left_fit array
                else:
                    if x1> right_region_boundary and x2> right_region_boundary: #Checks if the line is in the right region
                        right_fit.append((slope, intercept)) #Appends points to right_fit array
                        
        left_fit_average= np.average(left_fit,axis=0) #finds average of all the lines on the left region
        if len(left_fit)>0:
            lane_lines.append(make_points(frame,left_fit_average)) #Returns major lines on the left
        right_fit_average = np.average(right_fit, axis=0) ##finds average of all the lines on the right region
        if len(right_fit)>0:
            lane_lines.append(make_points(frame, right_fit_average)) #Returns major lines on the right
        logging.debug('lane lines: %s' % lane_lines)
        
    return lane_lines
    

In [11]:
def display_lines(frame, lines, line_color=(0, 255, 255), line_width=20):
    line_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image, (x1, y1), (x2, y2), line_color, line_width)
    line_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    return line_image


In [12]:
def draw_lines(img, lines, color=[0, 0, 255], thickness=3):
    # If there are no lines to draw, exit.
    if lines is None:
            return
    # Make a copy of the original image.
    img = np.copy(img)
    # Create a blank image that matches the original in size.
    line_img = np.zeros(
        (
            img.shape[0],
            img.shape[1],
            3
        ),
        dtype=np.uint8,
    )
    # Loop over all lines and draw them on the blank image.
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
    # Merge the image with the lines onto the original.
    img = cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0)
    # Return the modified image.
    return img

In [13]:
def display_heading_line(frame, steering_angle, line_color=(0, 0, 255), line_width=5 ):
    heading_image = np.zeros_like(frame)
    height, width, _ = frame.shape

    # figure out the heading line from steering angle
    # heading line (x1,y1) is always center bottom of the screen
    # (x2, y2) requires a bit of trigonometry

    # Note: the steering angle of:
    # 0-89 degree: turn left
    # 90 degree: going straight
    # 91-180 degree: turn right 
    steering_angle_radian = steering_angle / 180.0 * math.pi
    x1 = int(width / 2)
    y1 = height
    x2 = int(x1 - height / 2 / math.tan(steering_angle_radian))
    y2 = int(height / 2)

    cv2.line(heading_image, (x1, y1), (x2, y2), line_color, line_width)
    heading_image = cv2.addWeighted(frame, 0.8, heading_image, 1, 1)

    return heading_image

In [14]:
#Python code to interface Jetson Nano and Arduino 

import smbus


# Nvidia Jetson Nano I2C Bus 1
bus = smbus.SMBus(1)

 

# This is the address we setup in the Arduino Program(Slave Address)
address = 0x40

 
#Transmitted character value from Jetson Nano(Master) to Arduino Nano(Slave)
def writeNumber(value):
    bus.write_byte(address, value)
    # bus.write_byte_data(address, 1, value)
    return (0)

 
#Function defined to steer the bot in the right direction
def turn_func(stabilize_steering_angle):
    if stabilize_steering_angle>0 and stabilize_steering_angle<90:
        var=1
        writeNumber(var)
    if stabilize_steering_angle==90:
        var=2
        writeNumber(var)
    if stabilize_steering_angle>90 and stabilize_steering_angle<180:
        var=3
        writeNumber(var)


    




In [15]:
#Function defined to pass main directing parameters
def stabilize_steering_angle(
      curr_steering_angle,
      new_steering_angle,
      num_of_lane_lines,
      max_angle_deviation_two_lines=5,
      max_angle_deviation_one_lane=1):
  
    """
    Using last steering angle to stabilize the steering angle
    if new angle is too different from current angle,
    only turn by max_angle_deviation degrees
    """
    if num_of_lane_lines == 2 :
        # When both lane lines are detected more deviation occurs
        max_angle_deviation = max_angle_deviation_two_lines
    else :
        # When only one lane line is detected it does not deviate as much
        max_angle_deviation = max_angle_deviation_one_lane
        
    """
     This fragment takes the previous steering angle and compares it with 
     the current steering angle, if there is excessive variation then it
     uses the following process to change the current steering angle to within
     a set tolerance range. This is done to ensure the bot can turn smoothly when required.
     """
   
    angle_deviation = new_steering_angle - curr_steering_angle
    if abs(angle_deviation) > max_angle_deviation: 
        stabilized_steering_angle = int(curr_steering_angle
            + max_angle_deviation * angle_deviation / abs(angle_deviation))
    else:
        stabilized_steering_angle = new_steering_angle
    return stabilized_steering_angle

In [16]:



 # if you have multiple webcams change the value to the correct one


# Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.
#
# By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

# In[4]:

# What model to download.
MODEL_NAME = 'new_graph'  # change to whatever folder has the new graph
# MODEL_FILE = MODEL_NAME + '.tar.gz'   # these lines not needed as we are using our own model
# DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT =  '/home/jetson/TF-models/research/object_detection/new_graph/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/home/jetson/TF-models/research/object_detection/training/', 'object-detection.pbtxt')

NUM_CLASSES = 5  # we are only using one class in this example (cloth_mask)

# we don't need to download model since we have our own
# ## Download Model

# In[5]:
#
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#     file_name = os.path.basename(file.name)
#     if 'frozen_inference_graph.pb' in file_name:
#         tar_file.extract(file, os.getcwd())


# ## Load a (frozen) Tensorflow model into memory.

# In[6]:

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


# ## Loading label map
# Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

# In[7]:

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


# ## Helper code

# In[8]:

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)


# # Detection

# In[9]:

# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.


# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


# In[10]:
def get_prediction(image_np):
#     with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
#             while True:
                
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                    # Each box represents a part of the image where a particular object was detected.
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                    # Each score represent how level of confidence for each of the objects.
                    # Score is shown on the result image, together with the class label.
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
                    # Actual detection.
        (boxes, scores, classes, num_detections) = sess.run(
                        [boxes, scores, classes, num_detections],
                        feed_dict={image_tensor: image_np_expanded})
        return (boxes, scores, classes, num_detections)
                # Visualization of the results of a detection.
                
                

In [17]:
import ipywidgets
from IPython.display import display


image_widget=ipywidgets.Image(format='jpeg')

image_widget.value=bgr8_to_jpeg(image)
display(image_widget)
steer_array=[]

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [18]:
cam.running=True

def update_image(change):
    image=change["new"]
    kenerl_size=(3,3)
    gauss_image=cv2.GaussianBlur(image,kenerl_size,0) #gaussian blur is applied with kernel size 3x3
    hsv_image=cv2.cvtColor(gauss_image, cv2.COLOR_BGR2HSV) #RGB image is converted to HSV image
    lower_black= np.array([0,0,0]) 
    upper_black=np.array([227,277,226])
    thres_1=cv2.inRange(hsv_image, lower_black, upper_black) #HSV image is converted to binary image with limits for upper and lower black
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
    #thresh=cv2.dilate(thres_1, rectKernel, iterations=1)
    low_threshold =200
    high_threshold= 450
    canny_edges=cv2.Canny(thres_1, low_threshold, high_threshold) #canny edge detection is performed
    roi_image=region_of_interest(canny_edges) #only region of interest ie. lower half is considered
    line_segments= detect_line_segments(roi_image) #hough transform to detect line segments is performed
    lanes= average_slope_intercept(image, line_segments) #line segments are combined to return major lane lines
    line_image = display_lines(image, line_segments) #the returned lane lines are displayed on the image
    #image_widget.value=bgr8_to_jpeg(line_image)
    print(time.time())
    boxes, scores, classes, num_detections=get_prediction(image)
    print(time.time())

    try:
        if len(line_segments)==0 :
            return 0
        
    except: print()
 
    height, width,_= image.shape   
    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []
    #print("line_segments:" ,line_segments)
    if np.any(line_segments!=None):
        for line in line_segments:
            for x1, y1, x2, y2 in line:
                #if x1!=None and x2!=None and y1!=None and y2!=None:
                slope = (y2 - y1) / (x2 - x1) # <-- Calculating the slope.
                                #if math.fabs(slope) < 0.5: # <-- Only consider extreme slope
                                #   continue
                if slope <= 0: # <-- If the slope is negative, left group.
                    left_line_x.extend([x1, x2])
                    left_line_y.extend([y1, y2])
                else: # <-- Otherwise, right group.
                    right_line_x.extend([x1, x2])
                    right_line_y.extend([y1, y2])

    min_y = int(image.shape[0] * (2 / 5)) # <-- Just below the horizon
    max_y = image.shape[0] # <-- The bottom of the image

    if len(left_line_y)!=0 and len(left_line_x)!=0:         
        poly_left = np.poly1d(np.polyfit(
            left_line_y,
            left_line_x,
            deg=1
        ))
        left_x_start = int(poly_left(max_y))
        left_x_end = int(poly_left(min_y))
        #print("poly_left: ", poly_left)
    if len(right_line_x)!=0 and len(right_line_y)!=0 :
        poly_right = np.poly1d(np.polyfit(
            right_line_y,
            right_line_x,
            deg=1
        ))
        right_x_start = int(poly_right(max_y))
        right_x_end = int(poly_right(min_y))
        #print("poly_right: ", poly_right)
   
    if (len(right_line_x)==0 and len(right_line_y)==0) and (len(left_line_y)!=0 and len(left_line_x)!=0):   
        x1, x2 = left_x_start, left_x_end
        x_offset = x2 - x1
        y_offset = int(height / 2)
    if (len(left_line_x)==0 and len(left_line_y)==0) and (len(right_line_x)!=0 and len(right_line_y)!=0 ):   
        x1 , x2 = right_x_start,right_x_end
        x_offset = x2 - x1
        y_offset = int(height / 2)
    elif(len(right_line_x)!=0 and len(right_line_y)!=0 and len(left_line_y)!=0 and len(left_line_x)!=0):
        left_x2 = left_x_end
        right_x2 = right_x_end
        mid = int(width / 2)
        x_offset = (left_x2 + right_x2) / 2 - mid
        y_offset = int(height / 2)
        line_image = draw_lines( 
            image,
            [[
                [left_x_start, max_y, left_x_end, min_y],
                [right_x_start, max_y, right_x_end, min_y],
            ]],
            thickness=9,
        )

    
    if (len(left_line_x)!=0 and len(left_line_y)!=0) or (len(right_line_x)!=0 and len(right_line_y)!=0 ): 
   
        angle_to_mid_radian = math.atan(x_offset / y_offset)  # angle (in radian) to center vertical line
        angle_to_mid_deg = int(angle_to_mid_radian * 180.0 / math.pi)  # angle (in degrees) to center vertical line
        steering_angle = angle_to_mid_deg + 90  # this is the steering angle needed by picar front wheel 
        steer_array.append(steering_angle)
        stabilized_steering_angle = stabilize_steering_angle( steer_array[len(steer_array)-2], 
                                                              steer_array[len(steer_array)-1], 
                                                              2, 
                                                              max_angle_deviation_two_lines=5, 
                                                              max_angle_deviation_one_lane=1)
        steer_array[len(steer_array)-1]=stabilized_steering_angle
        
        steered_image=display_heading_line(line_image, stabilized_steering_angle, line_color=(0, 255, 0), line_width=5 )
        print("plot: ", time.time())
        vis_util.visualize_boxes_and_labels_on_image_array(
                    steered_image,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8)
        print("plotend: ", time.time())

    
        image_widget.value=bgr8_to_jpeg(steered_image)
    else:
        print("plot: ", time.time())
        vis_util.visualize_boxes_and_labels_on_image_array(
                    line_image,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8)
        image_widget.value=bgr8_to_jpeg(line_image)
        print("plotend: ", time.time())
        
    if len(steer_array)>4:
        steer_array.remove(steer_array[0])  
   
    

                #cv2.imshow('object detection', cv2.resize(image_np, (800, 600)))
                #cv2.imwrite("TF-models/research/object_detection/output_webcam/"+str(time.time())+".jpg", cv2.resize(image_np, (800, 600)))
    

    
    
cam.observe(update_image, names='value')
image_widget=ipywidgets.Image(format='jpeg')

#image_widget.value=bgr8_to_jpeg(image)
display(image_widget)

Image(value=b'', format='jpeg')

In [19]:
#cam.unobserve(update_image, names='value')

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1625843048.141938
1625843138.0699754
plot:  1625843138.3366933
plotend:  1625843138.3384538
1625843138.4439328
1625843142.3344033
plot:  1625843142.367661
plotend:  1625843142.369074
1625843142.398449
1625843145.728152
plot:  1625843145.737536
plotend:  1625843145.7392468
1625843145.7714412
1625843149.0458624
plot:  1625843149.0544329
plotend:  1625843149.276795
1625843149.5533257
1625843152.8073082
plot:  1625843152.8194597
plotend:  1625843152.8359375


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in int_scalars


1625843153.1488678
1625843156.4579413
plot:  1625843156.4691062
plotend:  1625843156.4854152
1625843156.7573576
1625843160.0675466
plot:  1625843160.0833256
plotend:  1625843160.0998106
1625843160.3765054
1625843163.6367722
plot:  1625843163.6480014
plotend:  1625843163.6642745
1625843163.955376
1625843167.199525
plot:  1625843167.2111766
plotend:  1625843167.2434106
1625843167.5395448
1625843170.903427
plot:  1625843170.9153998
plotend:  1625843170.9318511
1625843171.1993089
1625843174.428415
plot:  1625843174.4399586
plotend:  1625843174.4712715
1625843174.7505054
1625843178.103233
plot:  1625843178.1154864
plotend:  1625843178.1320302
1625843178.367901
1625843181.600629
plot:  1625843181.6115084
plotend:  1625843181.6279063
1625843181.738698
1625843184.9481158
plot:  1625843184.9577565
plotend:  1625843184.989288
1625843185.1178694
1625843188.3984544
plot:  1625843188.4084158
plotend:  1625843188.4401708
1625843188.55252
1625843191.7742815
plot:  1625843191.7839813
plotend:  1625843